In [84]:
import requests
import json
import csv
import pandas as pd

In [42]:
def get_url(layerDefs, returnIdsOnly = False):
    '''
        Function to get query url for rest api
    '''
    return f'https://arcgis.metc.state.mn.us/server/rest/services/EnterpriseLibrary/TransitStops/FeatureServer/query?layerDefs={layerDefs}&returnIdsOnly={str(returnIdsOnly).lower()}&f=json'

In [66]:
# Get all object ids at this endpoint
layerDefs = '{"0":"OBJECTID>0"}'
response = json.loads(requests.post(get_url(layerDefs, returnIdsOnly=True)).text)
objectIds = response['layers'][0]['objectIds']
len(objectIds)

20874

In [81]:
# Iterate over 1500 objects at a time (max limit: 2000)
stations = []
for i in range(int(len(objectIds) / 1500 + 1)):
    min = i * 1500
    max = (i + 1) * 1500
    layerDefs = '{"0":"OBJECTID>' + str(min) + ' AND OBJECTID<' + str(max) + '"}'
    object_dict = json.loads(requests.post(get_url(layerDefs, returnIdsOnly=False)).text)
    stations.extend([s['attributes'] for s in object_dict['layers'][0]['features']])

In [82]:
# Get columns
columns = {k for s in stations for k, v in s.items()}

In [83]:
# Save as CSV
with open('MN_station_metadata.csv', 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    csvwriter.writerow(columns)
    for station in stations:
        csvwriter.writerow([station[c] for c in columns])

In [86]:
metadata = pd.read_csv('MN_station_metadata.csv')
metadata.head()

,gate,dt_zone,site_at,publiccomm,parkride,UNIQUE_ID,last_edited_date,created_date,route_provider,last_edited_user,...,sidewalk,OBJECTID,created_user,adaaccess,county,active_status,site_on,GlobalID,busstop_yn,city
0,NaN,N,Entrance,#1145 Shenandoah Lane Main Entrance,N,5CE7DEF4-2B4E-4EC3-BF25-ADECC0C5A609,1648270552000,1648270552000,NaN,SVC-GISBATCH,...,NaN,1,SVC-GISBATCH,Y,Hennepin,Archive,#1145 Shenandoah Lane,{0067792E-A6C4-4459-AD31-8F717FD4D20F},Y,PLYMOUTH
1,NaN,N,Johnny Cake Ridge,Reservation Only,N,FED71E1C-0F8F-4E1B-9BD6-0AEEABB2468D,1648270552000,1648270552000,NaN,SVC-GISBATCH,...,NaN,2,SVC-GISBATCH,Y,Dakota,Archive,#12155 ESS - REZ,{E2CE868F-923F-4451-A3E8-CC362B3832AA},N,APPLE VALLEY
2,NaN,N,Hwy 35E,No bus svc - P&R #513 only,Y,3508011C-EF98-4878-ADDE-F6105F85D82B,1648270552000,1648270552000,NaN,SVC-GISBATCH,...,NaN,3,SVC-GISBATCH,N,Anoka,Archive,#2200 Main St P&R,{FF49CA3F-6EBA-42E0-A2A0-803F12C06066},N,LINO LAKES
3,NaN,N,Co Rd B-2 Fair P&R,2006 last used - was B2CL,N,0C22A85A-03BE-4CEC-B64E-B438545B12C9,1648270552000,1648270552000,NaN,SVC-GISBATCH,...,NaN,4,SVC-GISBATCH,Y,Ramsey,Archive,#2420 Cleveland,{4DDDFA09-3435-4D6E-8C98-35D96281D4E3},N,ROSEVILLE
4,NaN,N,Parklawn Ave,Entrance,N,5C9A078B-41C7-464F-AE7E-CC415BC61141,1648270552000,1648270552000,Met Council,SVC-GISBATCH,...,NaN,5,SVC-GISBATCH,Y,Hennepin,Active,#3400 Entrance,{FCAA3258-CECA-4B77-8E96-71745176126C},Y,EDINA
